# FaceForensics++ C23 – Face-Crop Pipeline and Training

This notebook builds a new pipeline that:

1. Loads `FF++_Metadata.csv` and train/val/test CSVs.
2. Extracts frames from the original videos and **crops faces** using MTCNN.
3. Saves cropped faces to `data_faces/{train,val,test}/{REAL,FAKE}`.
4. Trains a ResNet18 classifier on the cropped-face dataset.

Use this as an alternative to the full-frame pipeline in `main.ipynb`.


In [ ]:
# 1. Imports and basic paths
from pathlib import Path
import os

import cv2
import pandas as pd
from tqdm import tqdm
import torch

root = Path("FaceForensics++_C23")
print("Root:", root.resolve())

# Expect CSVs already created by main.ipynb (selected_videos + train/val/test)
print("Metadata exists:", (root / "csv" / "FF++_Metadata.csv").is_file())
print("Train CSV exists:", (root / "csv" / "train_videos.csv").is_file())
print("Val CSV exists:", (root / "csv" / "val_videos.csv").is_file())
print("Test CSV exists:", (root / "csv" / "test_videos.csv").is_file())

In [ ]:
# 2. Install and set up MTCNN face detector
# Run this cell once per environment; comment out pip line after install.

# !pip install facenet-pytorch

from facenet_pytorch import MTCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

mtcnn = MTCNN(keep_all=False, device=device)  # detect a single main face

In [ ]:
# 3. Helper: crop face from a single frame
import numpy as np


def crop_face_from_frame(frame_bgr, margin=0.2, target_size=(224, 224)):
    """Detect and crop the main face region from a BGR frame.

    Returns a BGR image of size target_size, or None if no face is found.
    """
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    h, w, _ = frame_rgb.shape

    boxes, _ = mtcnn.detect(frame_rgb)
    if boxes is None or len(boxes) == 0:
        return None

    # Take the first detected box (or we could choose the largest)
    x1, y1, x2, y2 = boxes[0]

    # Add a margin around the detected box
    bw = x2 - x1
    bh = y2 - y1
    x1 = max(0, int(x1 - margin * bw))
    y1 = max(0, int(y1 - margin * bh))
    x2 = min(w, int(x2 + margin * bw))
    y2 = min(h, int(y2 + margin * bh))

    if x2 <= x1 or y2 <= y1:
        return None

    face_rgb = frame_rgb[y1:y2, x1:x2]
    if face_rgb.size == 0:
        return None

    face_rgb = cv2.resize(face_rgb, target_size)
    face_bgr = cv2.cvtColor(face_rgb, cv2.COLOR_RGB2BGR)
    return face_bgr

In [ ]:
# 4. Frame index sampling (copied from main.ipynb for consistency)

def compute_sampled_indices(frame_count, assumed_fps=25, max_frames_per_video=40):
    """Return list of frame indices to sample given total frame_count.

    - step ~ 1 fps -> step = assumed_fps
    - start from `step` to avoid very first frames
    - cap the number of frames by max_frames_per_video
    """
    if frame_count <= 0:
        return []

    step = int(assumed_fps)
    if step <= 0:
        step = 1

    start = step  # skip the very first second
    indices = list(range(start, frame_count, step))

    if len(indices) > max_frames_per_video:
        indices = indices[:max_frames_per_video]
    return indices


# Quick sanity checks
for F in [300, 600, 1200]:
    idx = compute_sampled_indices(F)
    print(f"F={F}, num_frames={len(idx)}, first={idx[:3]} last={idx[-3:] if idx else []}")

In [ ]:
# 5. Extract face crops for each split

faces_root = Path("data_faces")
faces_root.mkdir(exist_ok=True)


def extract_face_frames_for_split(
    split_name,
    csv_name,
    max_frames_per_video=40,
    assumed_fps=25,
    target_size=(224, 224),
):
    """Extract sampled frames for all videos in the given split,
    detect and crop face, and save to data_faces/{split}/{REAL|FAKE}.
    """
    split_root = faces_root / split_name
    (split_root / "REAL").mkdir(parents=True, exist_ok=True)
    (split_root / "FAKE").mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(root / "csv" / csv_name)

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Extracting {split_name} faces"):
        rel_path = row["File Path"]
        label = row["Label"]  # 'REAL' or 'FAKE'
        frame_count = int(row.get("Frame Count", -1))

        video_path = root / rel_path
        if not video_path.is_file():
            continue

        indices = compute_sampled_indices(
            frame_count,
            assumed_fps=assumed_fps,
            max_frames_per_video=max_frames_per_video,
        )
        if not indices:
            continue

        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            continue

        base_name = video_path.stem

        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if not ret:
                continue

            face = crop_face_from_frame(frame, target_size=target_size)
            if face is None:
                continue

            out_dir = split_root / label
            frame_filename = f"{base_name}_f{idx:05d}.png"
            out_path = out_dir / frame_filename
            cv2.imwrite(str(out_path), face)

        cap.release()


print("data_faces root:", faces_root.resolve())
print("To run extraction, call extract_face_frames_for_split for train/val/test.")

In [ ]:
# 6. Run extraction (WARNING: long-running)

# Uncomment and run one by one when ready (after Kaggle download is complete).
# extract_face_frames_for_split("train", "train_videos.csv")
# extract_face_frames_for_split("val", "val_videos.csv")
# extract_face_frames_for_split("test", "test_videos.csv")

## Load cropped-face dataset and train ResNet18

After running the extraction above, `data_faces/` will contain:

- `data_faces/train/REAL` and `data_faces/train/FAKE`
- `data_faces/val/REAL` and `data_faces/val/FAKE`
- `data_faces/test/REAL` and `data_faces/test/FAKE`

We can now reuse the same training code structure as in `main.ipynb`, just pointing to `data_faces` instead of `data_frames`.

In [ ]:
# 7. ImageFolder datasets and DataLoaders for data_faces

from torch.utils.data import DataLoader
from torchvision import datasets, transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

faces_dir = Path("data_faces")

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.08),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


eval_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


train_dataset = datasets.ImageFolder(root=str(faces_dir / "train"), transform=train_transform)
val_dataset = datasets.ImageFolder(root=str(faces_dir / "val"), transform=eval_transform)
test_dataset = datasets.ImageFolder(root=str(faces_dir / "test"), transform=eval_transform)

print("Classes:", train_dataset.classes)
print("Train images:", len(train_dataset))
print("Val images:", len(val_dataset))
print("Test images:", len(test_dataset))

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

len(train_loader), len(val_loader), len(test_loader)

In [ ]:
# 8. Define ResNet18 model, loss, optimizer (similar to main.ipynb)

import torch.nn as nn
from torchvision import models


# If you want to reuse class_weights from main.ipynb, you can paste them here manually.
# For simplicity, we start with unweighted loss; you can add weights later if needed.

cw = None


def create_model(num_classes=2, use_pretrained=True):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if use_pretrained else None)
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features, num_classes),
    )
    return model


model = create_model(num_classes=2, use_pretrained=True).to(device)

criterion = nn.CrossEntropyLoss(weight=cw)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1)

print(model.fc)
criterion, optimizer, scheduler

In [ ]:
# 9. Training and validation loop (copied from main.ipynb)

import time


def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += inputs.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += inputs.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc


num_epochs = 20
best_val_acc = 0.0
best_state_dict = None

history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(num_epochs):
    start_time = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    elapsed = time.time() - start_time

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    scheduler.step(val_loss)

    print(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
        f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}, "
        f"time={elapsed:.1f}s"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state_dict = model.state_dict()

print("Best val acc:", best_val_acc)

if best_state_dict is not None:
    model.load_state_dict(best_state_dict)
    torch.save(model.state_dict(), "best_resnet_ffpp_faces_binary.pth")
    print("Saved best model to best_resnet_ffpp_faces_binary.pth")

In [ ]:
# 10. Test evaluation and plots

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


@torch.no_grad()
def evaluate_simple(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += inputs.size(0)
    return correct / total if total > 0 else 0.0


test_acc = evaluate_simple(model, test_loader, device)
print("Test accuracy (faces):", test_acc)

# Plot training and validation loss/accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss over epochs (faces)")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history["train_acc"], label="Train Acc")
plt.plot(history["val_acc"], label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy over epochs (faces)")
plt.legend()

plt.tight_layout()
plt.show()


@torch.no_grad()
def get_predictions_and_labels(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    return all_preds, all_labels


y_pred, y_true = get_predictions_and_labels(model, test_loader, device)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix (rows=true, cols=pred):")
print(cm)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes, digits=4))

## Day Experiment – Face Crops from Existing `data_frames`

This section builds a quicker experiment:

- Input: already preprocessed full-frame images in `data_frames/{train,val,test}/{REAL,FAKE}`.
- Step: run MTCNN on each 224×224 image, crop face, and save to `data_faces_from_frames/...`.
- Then: train the same ResNet18 model on this new dataset to see if overfitting improves.

Use this to get faster feedback during the day before running the full raw-video face-crop pipeline overnight.

In [ ]:
# 11. Paths for existing frames and new face-crops-from-frames

from pathlib import Path
import os

import cv2
from tqdm import tqdm

# Existing full-frame images from main.ipynb
frames_root = Path("data_frames")  # already contains train/val/test/REAL,FAKE
print("Existing frames root:", frames_root.resolve())

faces_from_frames_root = Path("data_faces_from_frames")
faces_from_frames_root.mkdir(exist_ok=True)
print("Faces-from-frames root:", faces_from_frames_root.resolve())

In [ ]:
# 12. Helper: crop faces from a folder of existing 224x224 frames

import numpy as np


def crop_faces_in_split_from_frames(split_name, max_files_per_class=None):
    """For a given split (train/val/test),
    read images from data_frames/{split}/{REAL,FAKE},
    run MTCNN to detect/crop face, and save to
    data_faces_from_frames/{split}/{REAL,FAKE}.

    max_files_per_class: optional cap on number of images per class
    (for quicker experiments).
    """
    src_split = frames_root / split_name
    dst_split = faces_from_frames_root / split_name

    for cls in ["REAL", "FAKE"]:
        src_dir = src_split / cls
        dst_dir = dst_split / cls
        dst_dir.mkdir(parents=True, exist_ok=True)

        if not src_dir.exists():
            print(f"Warning: source dir not found: {src_dir}")
            continue

        files = [p for p in src_dir.iterdir() if p.is_file()]
        if max_files_per_class is not None:
            files = files[: max_files_per_class]

        print(f"[{split_name}] {cls}: {len(files)} images to process")

        for img_path in tqdm(files, desc=f"{split_name}-{cls}"):
            frame_bgr = cv2.imread(str(img_path))
            if frame_bgr is None:
                continue

            face_bgr = crop_face_from_frame(frame_bgr, target_size=(224, 224))
            if face_bgr is None:
                continue

            out_path = dst_dir / img_path.name
            cv2.imwrite(str(out_path), face_bgr)

    print(f"Done processing split: {split_name}")

In [ ]:
# 13. Run face-cropping from existing frames (quick experiment)

# You can set max_files_per_class to a smaller number (e.g., 200)
# for a very fast test, or None to process all images in each class.

# Example: quick test on 200 images per class
# crop_faces_in_split_from_frames("train", max_files_per_class=200)
# crop_faces_in_split_from_frames("val", max_files_per_class=200)
# crop_faces_in_split_from_frames("test", max_files_per_class=200)

# Example: full run on all available images (may take longer)
# crop_faces_in_split_from_frames("train", max_files_per_class=None)
# crop_faces_in_split_from_frames("val", max_files_per_class=None)
# crop_faces_in_split_from_frames("test", max_files_per_class=None)

### Train on `data_faces_from_frames` (day experiment)

After running the cropping above, the quick-experiment dataset will be in:

- `data_faces_from_frames/train/{REAL,FAKE}`
- `data_faces_from_frames/val/{REAL,FAKE}`
- `data_faces_from_frames/test/{REAL,FAKE}`

We can reuse the same training code structure, just pointing to this new root.

In [ ]:
# 14. DataLoaders for data_faces_from_frames (quick experiment)

from torch.utils.data import DataLoader
from torchvision import datasets, transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device (day experiment):", device)

faces_ff_root = Path("data_faces_from_frames")

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transform_ff = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.08),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


eval_transform_ff = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


train_dataset_ff = datasets.ImageFolder(root=str(faces_ff_root / "train"), transform=train_transform_ff)
val_dataset_ff = datasets.ImageFolder(root=str(faces_ff_root / "val"), transform=eval_transform_ff)
test_dataset_ff = datasets.ImageFolder(root=str(faces_ff_root / "test"), transform=eval_transform_ff)

print("[from_frames] Classes:", train_dataset_ff.classes)
print("[from_frames] Train images:", len(train_dataset_ff))
print("[from_frames] Val images:", len(val_dataset_ff))
print("[from_frames] Test images:", len(test_dataset_ff))

batch_size = 64

train_loader_ff = DataLoader(train_dataset_ff, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader_ff = DataLoader(val_dataset_ff, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader_ff = DataLoader(test_dataset_ff, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

len(train_loader_ff), len(val_loader_ff), len(test_loader_ff)

In [ ]:
# 15. Model and training for data_faces_from_frames (reuse same architecture)

import torch.nn as nn
from torchvision import models
import time


cw_ff = None  # can plug class weights here if desired


def create_model_ff(num_classes=2, use_pretrained=True):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if use_pretrained else None)
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features, num_classes),
    )
    return model


model_ff = create_model_ff(num_classes=2, use_pretrained=True).to(device)

criterion_ff = nn.CrossEntropyLoss(weight=cw_ff)
optimizer_ff = torch.optim.Adam(model_ff.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler_ff = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ff, mode="min", factor=0.5, patience=1)


def train_one_epoch_ff(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += inputs.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc


@torch.no_grad()
def evaluate_ff(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += inputs.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc


num_epochs_ff = 15
best_val_acc_ff = 0.0
best_state_dict_ff = None

history_ff = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(num_epochs_ff):
    start_time = time.time()
    train_loss, train_acc = train_one_epoch_ff(model_ff, train_loader_ff, optimizer_ff, criterion_ff, device)
    val_loss, val_acc = evaluate_ff(model_ff, val_loader_ff, criterion_ff, device)
    elapsed = time.time() - start_time

    history_ff["train_loss"].append(train_loss)
    history_ff["train_acc"].append(train_acc)
    history_ff["val_loss"].append(val_loss)
    history_ff["val_acc"].append(val_acc)

    scheduler_ff.step(val_loss)

    print(
        f"[from_frames] Epoch {epoch+1}/{num_epochs_ff} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
        f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}, "
        f"time={elapsed:.1f}s"
    )

    if val_acc > best_val_acc_ff:
        best_val_acc_ff = val_acc
        best_state_dict_ff = model_ff.state_dict()

print("[from_frames] Best val acc:", best_val_acc_ff)

if best_state_dict_ff is not None:
    model_ff.load_state_dict(best_state_dict_ff)
    torch.save(model_ff.state_dict(), "best_resnet_ffpp_faces_from_frames.pth")
    print("Saved best model to best_resnet_ffpp_faces_from_frames.pth")

In [ ]:
# 16. Evaluation and plots for data_faces_from_frames (day experiment)

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


@torch.no_grad()
def evaluate_simple_ff(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += inputs.size(0)
    return correct / total if total > 0 else 0.0


test_acc_ff = evaluate_simple_ff(model_ff, test_loader_ff, device)
print("[from_frames] Test accuracy:", test_acc_ff)

# Plot training and validation loss/accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_ff["train_loss"], label="Train Loss")
plt.plot(history_ff["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss over epochs (faces_from_frames)")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_ff["train_acc"], label="Train Acc")
plt.plot(history_ff["val_acc"], label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy over epochs (faces_from_frames)")
plt.legend()

plt.tight_layout()
plt.show()


@torch.no_grad()
def get_predictions_and_labels_ff(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    return all_preds, all_labels


y_pred_ff, y_true_ff = get_predictions_and_labels_ff(model_ff, test_loader_ff, device)

cm_ff = confusion_matrix(y_true_ff, y_pred_ff)
print("[from_frames] Confusion Matrix (rows=true, cols=pred):")
print(cm_ff)

print("\n[from_frames] Classification Report:")
print(classification_report(y_true_ff, y_pred_ff, target_names=train_dataset_ff.classes, digits=4))